In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Load the dataset
a1=pd.read_csv('/content/case_study1.xlsx - case_study1.csv')
a2=pd.read_csv('/content/case_study2.xlsx - case_study2.csv')

In [ ]:
a1.shape,a2.shape

((51336, 26), (51336, 62))

In [ ]:
df1=a1.copy()
df2=a2.copy()

In [ ]:
columns_to_be_removed = []

for i in df2.columns:
  if (df2.loc[df2[i]==-99999].shape[0]>10000):
    columns_to_be_removed.append(i)



In [ ]:
columns_to_be_removed

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [ ]:
df2.drop(columns=columns_to_be_removed,axis=1,inplace=True)

In [ ]:
df2.shape

(51336, 54)

In [ ]:
for i in df2.columns:
  df2=df2.loc[df2[i]!=-99999]


In [ ]:
df2.shape

(42066, 54)

In [ ]:
# Remove nulls
df1=df1.loc[df1['Age_Oldest_TL'] != -99999]

In [ ]:
df1.shape

(51296, 26)

In [ ]:
#checking for comman columns
for i in df2.columns:
  if i in df1.columns:
    print(i)

PROSPECTID


In [ ]:
df1.isna().sum().sum()

0

In [ ]:
df2.isna().sum().sum()

0

In [ ]:
df=pd.merge(left=df1,right=df2,how='inner',left_on='PROSPECTID',right_on='PROSPECTID')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [ ]:
df.shape

(42064, 79)

In [ ]:
#check how many columns are categorical
categorical_cols=(df.select_dtypes(include='O').columns)
print(categorical_cols)

Index(['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2',
       'first_prod_enq2', 'Approved_Flag'],
      dtype='object')


In [ ]:
#chi_square test
for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2', 'Approved_Flag']:
  chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
  print(i, '---', pval)


MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287
Approved_Flag --- 0.0


In [ ]:
#VIF numerical columns
numerical_columns=df.select_dtypes(exclude='O').columns.tolist()[1:]

In [ ]:
vif_data = df[numerical_columns]
total_columns = vif_data.shape[1]
print(total_columns)
columns_to_be_kept = []
column_index = 0



for i in range (0,total_columns):

    vif_value = variance_inflation_factor(vif_data, column_index)
    print (column_index,'---',vif_value)


    if vif_value <= 6:
        columns_to_be_kept.append( numerical_columns[i] )
        column_index = column_index+1

    else:
        vif_data = vif_data.drop([ numerical_columns[i] ] , axis=1)

72


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf
0 --- 11.320180023967996
0 --- 8.363698035000336
0 --- 6.520647877790928
0 --- 5.149501618212625
1 --- 2.611111040579735


...

2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477228
2 --- 3.832800792153077
3 --- 6.099653381646723
3 --- 5.581352009642766
4 --- 1.985584353098778


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 --- inf
5 --- 4.80953830281934
6 --- 23.270628983464636
6 --- 30.595522588100053
6 --- 4.384346405965583
7 --- 3.0646584155234238
8 --- 2.898639771299251
9 --- 4.377876915347324
10 --- 2.207853583695844
11 --- 4.916914200506864
12 --- 5.214702030064725
13 --- 3.3861625024231476
14 --- 7.840583309478997
14 --- 5.255034641721434


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 --- inf
15 --- 7.380634506427238
15 --- 1.4210050015175733
16 --- 8.083255010190316
16 --- 1.6241227524040114
17 --- 7.257811920140003
17 --- 15.59624383268298
17 --- 1.825857047132431
18 --- 1.5080839450032664
19 --- 2.172088834824578
20 --- 2.6233975535272274
21 --- 2.2959970812106176
22 --- 7.360578319196446
22 --- 2.1602387773102567
23 --- 2.8686288267891467
24 --- 6.458218003637272
24 --- 2.8474118865638247
25 --- 4.753198156284083
26 --- 16.22735475594825
26 --- 6.424377256363877
26 --- 8.887080381808678
26 --- 2.3804746142952653
27 --- 8.60951347651454
27 --- 13.06755093547673
27 --- 3.500040056654653
28 --- 1.9087955874813773
29 --- 17.006562234161628
29 --- 10.730485153719197
29 --- 2.3538497522950275
30 --- 22.10485591513649
30 --- 2.7971639638512924
31 --- 3.424171203217696
32 --- 10.175021454450922
32 --- 6.408710354561292
32 --- 1.001151196262563
33 --- 3.069197305397273
34 --- 2.8091261600643724
35 --- 20.249538381980678
35 --- 15.864576541593774
35 --- 1.8331649740532

In [ ]:
(columns_to_be_kept)

['pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'Tot_TL_closed_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'CC_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'max_recent_level_of_deliq',
 'num_deliq_6_12mts',
 'num_times_60p_dpd',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_12mts',
 'num_lss',
 'num_lss_12mts',
 'recent_level_of_deliq',
 'CC_enq_L12m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L3m',
 'NETMONTHLYINCOME',
 'Time_With_Curr_Empr',
 'pct_currentBal_all_TL',
 'CC_Flag',
 'PL_Flag',
 'pct_PL_enq_L6m_of_ever',
 'pct_CC_enq_L6m_of_ever',
 'HL_Flag',
 'GL_Flag']

In [ ]:
# check Anova for columns_to_be_kept

from scipy.stats import f_oneway
columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
  a=list(df[i])
  b=list(df['Approved_Flag'])

  group_P1=[value for value,group in zip(a,b) if group == 'P1']
  group_P2=[value for value,group in zip(a,b) if group == 'P2']
  group_P3=[value for value,group in zip(a,b) if group == 'P3']
  group_P4=[value for value,group in zip(a,b) if group == 'P4']

  f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)
  if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)


In [ ]:
len(columns_to_be_kept_numerical)

37

In [ ]:
features=list(columns_to_be_kept_numerical)+list(categorical_cols)
len(features)

43

In [ ]:
len(df.columns)

79

In [ ]:
new_df=df[features]
new_df.columns

Index(['pct_tl_open_L6M', 'pct_tl_closed_L6M', 'Tot_TL_closed_L12M',
       'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL', 'Home_TL', 'PL_TL',
       'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL',
       'Age_Newest_TL', 'time_since_recent_payment',
       'max_recent_level_of_deliq', 'num_deliq_6_12mts', 'num_times_60p_dpd',
       'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt',
       'num_dbt_12mts', 'num_lss', 'recent_level_of_deliq', 'CC_enq_L12m',
       'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m', 'NETMONTHLYINCOME',
       'Time_With_Curr_Empr', 'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever',
       'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag', 'MARITALSTATUS',
       'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2',
       'Approved_Flag'],
      dtype='object')

In [ ]:
new_df

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2,Approved_Flag
0,0.000,0.00,0,0.000,0,0,0,4,1,4,...,0.000,0.0,1,0,Married,12TH,M,PL,PL,P2
1,0.000,0.00,0,0.000,0,0,0,0,0,1,...,0.000,0.0,0,0,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan,P2
2,0.125,0.00,0,0.000,1,0,0,0,2,6,...,0.000,0.0,1,0,Married,SSC,M,ConsumerLoan,others,P2
3,0.000,0.00,0,0.000,0,0,0,0,3,0,...,0.000,0.0,0,0,Married,POST-GRADUATE,M,AL,AL,P1
4,0.000,0.00,1,0.167,0,0,0,0,6,0,...,0.429,0.0,1,0,Married,12TH,M,ConsumerLoan,PL,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,0.333,0.00,0,0.000,0,0,0,0,0,3,...,0.000,0.0,0,0,Married,12TH,M,ConsumerLoan,ConsumerLoan,P4
42060,0.000,0.25,1,0.250,0,0,0,0,2,2,...,0.000,0.0,0,0,Married,SSC,M,others,others,P1
42061,0.500,0.50,1,0.500,0,0,0,0,0,2,...,1.000,0.0,0,0,Married,SSC,M,ConsumerLoan,others,P3
42062,0.000,0.00,1,0.500,0,0,0,0,0,2,...,0.000,0.0,0,0,Single,UNDER GRADUATE,F,ConsumerLoan,others,P2


In [ ]:
 #['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2', 'Approved_Flag']
for col in  ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2', 'Approved_Flag']:
  unique_counts=np.unique(new_df[col])
  print(f"{col}: {unique_counts}")

MARITALSTATUS: ['Married' 'Single']
EDUCATION: ['12TH' 'GRADUATE' 'OTHERS' 'POST-GRADUATE' 'PROFESSIONAL' 'SSC'
 'UNDER GRADUATE']
GENDER: ['F' 'M']
last_prod_enq2: ['AL' 'CC' 'ConsumerLoan' 'HL' 'PL' 'others']
first_prod_enq2: ['AL' 'CC' 'ConsumerLoan' 'HL' 'PL' 'others']
Approved_Flag: ['P1' 'P2' 'P3' 'P4']


In [ ]:
#['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
#'PROFESSIONAL']
new_df.loc[new_df['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
new_df.loc[new_df['EDUCATION'] == '12TH',['EDUCATION']]             = 2
new_df.loc[new_df['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 3
new_df.loc[new_df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
new_df.loc[new_df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 3
new_df.loc[new_df['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
new_df.loc[new_df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 4


In [ ]:
new_df['EDUCATION'].value_counts()

EDUCATION
3    18931
2    11703
1     9532
4     1898
Name: count, dtype: int64

In [ ]:
new_df['EDUCATION'] = new_df['EDUCATION'].astype(int)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            42064 non-null  float64
 1   pct_tl_closed_L6M          42064 non-null  float64
 2   Tot_TL_closed_L12M         42064 non-null  int64  
 3   pct_tl_closed_L12M         42064 non-null  float64
 4   Tot_Missed_Pmnt            42064 non-null  int64  
 5   CC_TL                      42064 non-null  int64  
 6   Home_TL                    42064 non-null  int64  
 7   PL_TL                      42064 non-null  int64  
 8   Secured_TL                 42064 non-null  int64  
 9   Unsecured_TL               42064 non-null  int64  
 10  Other_TL                   42064 non-null  int64  
 11  Age_Oldest_TL              42064 non-null  int64  
 12  Age_Newest_TL              42064 non-null  int64  
 13  time_since_recent_payment  42064 non-null  int

<ipython-input-34-bae01f794a9c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['EDUCATION'] = new_df['EDUCATION'].astype(int)


In [ ]:
df_encoded = pd.get_dummies(new_df, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])


In [ ]:
df_encoded.info()
k = df_encoded.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

# Machine Learing model fitting

# Data processing

# 1. Random Forest

In [ ]:
y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_recall_fscore_support
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
rf_classifier = RandomForestClassifier(n_estimators = 200, random_state=42)
rf_classifier.fit(x_train, y_train)
y_pred = rf_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy}')
print ()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)
for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.7636990372043266

Class p1:
Precision: 0.8370457209847597
Recall: 0.7041420118343196
F1 Score: 0.7648634172469203

Class p2:
Precision: 0.7957519116397621
Recall: 0.9282457879088206
F1 Score: 0.8569075937785909

Class p3:
Precision: 0.4423380726698262
Recall: 0.21132075471698114
F1 Score: 0.28600612870275793

Class p4:
Precision: 0.7178502879078695
Recall: 0.7269193391642371
F1 Score: 0.7223563495895703



#xgboost

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)



y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)




xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy:.2f}')
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()




Accuracy: 0.78

Class p1:
Precision: 0.823906083244397
Recall: 0.7613412228796844
F1 Score: 0.7913890312660173

Class p2:
Precision: 0.8255418233924413
Recall: 0.913577799801784
F1 Score: 0.8673315769665036

Class p3:
Precision: 0.4756380510440835
Recall: 0.30943396226415093
F1 Score: 0.3749428440786465

Class p4:
Precision: 0.7342386032977691
Recall: 0.7356656948493683
F1 Score: 0.7349514563106796



#Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier


y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)
dt_model.fit(x_train, y_train)
y_pred = dt_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f"Accuracy: {accuracy:.2f}")
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.71

Class p1:
Precision: 0.7205882352941176
Recall: 0.7248520710059172
F1 Score: 0.7227138643067846

Class p2:
Precision: 0.8096348096348096
Recall: 0.8261645193260654
F1 Score: 0.817816148337094

Class p3:
Precision: 0.3448275862068966
Recall: 0.32452830188679244
F1 Score: 0.33437013996889586

Class p4:
Precision: 0.6503006012024048
Recall: 0.630709426627794
F1 Score: 0.6403552047360631



In [ ]:
# # Hyperparameter tuning for xgboost (Used in the session)

# # Define the hyperparameter grid
param_grid = {
   'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 0.9],
   'learning_rate'   : [0.001, 0.01, 0.1, 1],
   'max_depth'       : [3, 5, 8, 10],
   'alpha'           : [1, 10, 100],
  'n_estimators'    : [10,50,100]
 }

index = 0

answers_grid = {
     'combination'       :[],
     'train_Accuracy'    :[],
     'test_Accuracy'     :[],
     'colsample_bytree'  :[],
     'learning_rate'     :[],
     'max_depth'         :[],
     'alpha'             :[],
     'n_estimators'      :[]

     }


# # Loop through each combination of hyperparameters
for colsample_bytree in param_grid['colsample_bytree']:
   for learning_rate in param_grid['learning_rate']:
    for max_depth in param_grid['max_depth']:
      for alpha in param_grid['alpha']:
          for n_estimators in param_grid['n_estimators']:

              index = index + 1

#               # Define and train the XGBoost model
              model = xgb.XGBClassifier(objective='multi:softmax',
                                        num_class=4,
                                       colsample_bytree = colsample_bytree,
                                        learning_rate = learning_rate,
                                        max_depth = max_depth,
                                        alpha = alpha,
                                        n_estimators = n_estimators)



              y = df_encoded['Approved_Flag']
              x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

              label_encoder = LabelEncoder()
              y_encoded = label_encoder.fit_transform(y)


              x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)


              model.fit(x_train, y_train)



#               # Predict on training and testing sets
              y_pred_train = model.predict(x_train)
              y_pred_test = model.predict(x_test)


#               # Calculate train and test results


              train_accuracy =  accuracy_score (y_train, y_pred_train)
              test_accuracy  =  accuracy_score (y_test , y_pred_test)



#               # Include into the lists
              answers_grid ['combination']   .append(index)
              answers_grid ['train_Accuracy']    .append(train_accuracy)
              answers_grid ['test_Accuracy']     .append(test_accuracy)
              answers_grid ['colsample_bytree']   .append(colsample_bytree)
              answers_grid ['learning_rate']      .append(learning_rate)
              answers_grid ['max_depth']          .append(max_depth)
              answers_grid ['alpha']              .append(alpha)
              answers_grid ['n_estimators']       .append(n_estimators)


#               # Print results for this combination
              print(f"Combination {index}")
              print(f"colsample_bytree: {colsample_bytree}, learning_rate: {learning_rate}, max_depth: {max_depth}, alpha: {alpha}, n_estimators: {n_estimators}")
              print(f"Train Accuracy: {train_accuracy:.2f}")
              print(f"Test Accuracy : {test_accuracy :.2f}")
              print("-" * 30)

Combination 1
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 10
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 2
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 50
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 3
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 100
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 4
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 10
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 5
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 50
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 6
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 100
Train Accuracy: 0.61
Test Accu

In [ ]:
pd.DataFrame(answers_grid).to_csv('Parameters.csv')

In [ ]:
(x_train.columns)


Index(['pct_tl_open_L6M', 'pct_tl_closed_L6M', 'Tot_TL_closed_L12M',
       'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL', 'Home_TL', 'PL_TL',
       'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL',
       'Age_Newest_TL', 'time_since_recent_payment',
       'max_recent_level_of_deliq', 'num_deliq_6_12mts', 'num_times_60p_dpd',
       'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt',
       'num_dbt_12mts', 'num_lss', 'recent_level_of_deliq', 'CC_enq_L12m',
       'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m', 'NETMONTHLYINCOME',
       'Time_With_Curr_Empr', 'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever',
       'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag', 'MARITALSTATUS',
       'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'],
      dtype='object')

In [ ]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.4.2
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: bigframes, fastai, imbalanced-learn, librosa, mlxtend, qudida, sklearn-pandas, yellowbrick


In [ ]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

trf=ColumnTransformer([
    ('ohe',OneHotEncoder(sparse_output=False),[37, 39, 40 ,41])
],remainder='passthrough',verbose_feature_names_out=False)


pipeline=Pipeline([
    ('trf',trf),
    ('xgb_classifier',xgb.XGBClassifier(objective='multi:softmax',  num_class=4,
                                   colsample_bytree=0.9,learning_rate=1,
                                   max_depth=10,alpha=10,n_estimators=10))
])



y = new_df['Approved_Flag']
x = new_df. drop ( ['Approved_Flag'], axis = 1 )


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)




pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy:.2f}')
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()



Accuracy: 0.77

Class p1:
Precision: 0.819047619047619
Recall: 0.7633136094674556
F1 Score: 0.7901990811638592

Class p2:
Precision: 0.8234871610702101
Recall: 0.9090188305252725
F1 Score: 0.8641416996419823

Class p3:
Precision: 0.4577625570776256
Recall: 0.3026415094339623
F1 Score: 0.364379827351204

Class p4:
Precision: 0.7165200391006843
Recall: 0.7123420796890184
F1 Score: 0.7144249512670566



In [ ]:
from  joblib import dump
# Dumping the object to a file
dump(pipeline, 'pipeline.joblib')


['pipeline.joblib']

In [ ]:
new_point={
  "pct_tl_open_L6M": 0.677,
  "pct_tl_closed_L6M": 0,
  "Tot_TL_closed_L12M": 0,
  "pct_tl_closed_L12M": 0,
  "Tot_Missed_Pmnt": 2,
  "CC_TL": 0,
  "Home_TL": 0,
  "PL_TL": 1,
  "Secured_TL": 2,
  "Unsecured_TL": 1,
  "Other_TL": 0,
  "Age_Oldest_TL": 18,
  "Age_Newest_TL": 2,
  "time_since_recent_payment": 0,
  "max_recent_level_of_deliq": 0,
  "num_deliq_6_12mts": 0,
  "num_times_60p_dpd": 0,
  "num_std_12mts": 2,
  "num_sub": 0,
  "num_sub_6mts": 0,
  "num_sub_12mts": 0,
  "num_dbt": 0,
  "num_dbt_12mts": 0,
  "num_lss": 0,
  "recent_level_of_deliq": 0,
  "CC_enq_L12m": 0,
  "PL_enq_L12m": 1,
  "time_since_recent_enq": 46,
  "enq_L3m": 1,
  "NETMONTHLYINCOME": 150000,
  "Time_With_Curr_Empr": 154,
  "CC_Flag": 0,
  "PL_Flag": 1,
  "pct_PL_enq_L6m_of_ever": 1.0,
  "pct_CC_enq_L6m_of_ever": 0,
  "HL_Flag": 0,
  "GL_Flag": 0,
  "MARITALSTATUS": "Single",
  "EDUCATION": 3,
  "GENDER": "F",
  "last_prod_enq2": "PL",
  "first_prod_enq2": "others"
}

In [ ]:
features=[0.677, 0.0, 0, 0.0, 2, 0, 0, 1, 2, 1, 0, 18, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 46, 1, 150000, 154, 0, 1, 1.0, 0.0, 0, 0, 'Single', 3, 'F', 'PL', 'others']
new_point=pd.DataFrame(data=[features],
                       columns=new_df.columns[:-1])

In [ ]:
new_point


,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2
0,0.677,0.0,0,0.0,2,0,0,1,2,1,...,1,1.0,0.0,0,0,Single,3,F,PL,others


In [ ]:
from joblib import load

pipeline_path = 'pipeline.pkl'

pipeline=load(pipeline_path)




In [ ]:
pipeline.predict(new_point)

array([1], dtype=int32)

In [ ]:
pipeline[:-1].get_feature_names_out()

array(['MARITALSTATUS_Married', 'MARITALSTATUS_Single', 'GENDER_F',
       'GENDER_M', 'last_prod_enq2_AL', 'last_prod_enq2_CC',
       'last_prod_enq2_ConsumerLoan', 'last_prod_enq2_HL',
       'last_prod_enq2_PL', 'last_prod_enq2_others', 'first_prod_enq2_AL',
       'first_prod_enq2_CC', 'first_prod_enq2_ConsumerLoan',
       'first_prod_enq2_HL', 'first_prod_enq2_PL',
       'first_prod_enq2_others', 'pct_tl_open_L6M', 'pct_tl_closed_L6M',
       'Tot_TL_closed_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt',
       'CC_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL',
       'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL',
       'time_since_recent_payment', 'max_recent_level_of_deliq',
       'num_deliq_6_12mts', 'num_times_60p_dpd', 'num_std_12mts',
       'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt',
       'num_dbt_12mts', 'num_lss', 'recent_level_of_deliq', 'CC_enq_L12m',
       'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m',
       'NETMONTHLYINCOME', 'Time_With

In [ ]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.4.2
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: bigframes, fastai, imbalanced-learn, librosa, mlxtend, qudida, sklearn-pandas, yellowbrick
